In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim


# 数据处理

In [2]:
class data_processing(object):
    def __init__(self,train_path,test_path):
        self.train_path=train_path
        self.test_path=test_path
    
    def to_num(self,dataset,test):
        list_str=[]
        for i in dataset.columns.tolist():
            if isinstance(dataset.loc[0,i],str):
                list_str.append(i)
        names=[]
        for str1 in list_str:
            name1=list(set(dataset.loc[:,str1].tolist()))
            names.append(name1)
        for i in range(len(list_str)):
            d=pd.DataFrame(np.where(np.array(dataset.loc[:,list_str[i]])[:,None]==np.array([names[i]]).repeat(len(\
                                            np.array(dataset.loc[:,list_str[i]])),axis=0))[1]).iloc[:,0]
            t=pd.DataFrame(np.where(np.array(test.loc[:,list_str[i]])[:,None]==np.array([names[i]]).repeat(len(\
                                            np.array(test.loc[:,list_str[i]])),axis=0))[1]).iloc[:,0]
            dataset.loc[:,list_str[i]]=d
            test.loc[:,list_str[i]]=t
        return dataset,test
    def stand(self,data_train,test):
        
        data_max=data_train.max(axis=0)
        data_min=data_train.min(axis=0)
        test_max=test.max(axis=0)
        test_min=test.min(axis=0)
        test=(test-test_min)/(test_max-test_min+1e-4)
        data_train=(data_train-data_min)/(data_max-data_min+1e-4)
        
        return data_train,test
    def norm(self,data_train,test):
        
        train_means=np.mean(data_train,axis=0)
        test_means=np.mean(test,axis=0)
        
        train_var=np.sqrt(np.float32(np.sum((data_train-train_means)**2,axis=0)))
        test_var=np.sqrt(np.float32(np.sum((test-test_means)**2,axis=0)))
        
        data_train=(data_train-train_means)/(train_var+1e-4)
        
        test=(test-test_means)/(test_var+1e-4)
        
        return data_train,test
    def load(self):
        
        datasets=pd.read_csv(self.train_path)
        test=pd.read_csv(self.test_path)
        dataset=pd.concat((datasets.iloc[:,1:13],datasets.iloc[:,14:]),axis=1)
        labels=np.array(datasets.iloc[:,13])
        dataset,test=self.to_num(dataset,test)
        dataset=np.array(dataset.fillna(0))
        test=np.array(test.fillna(0))
        
        dataset,test=self.stand(dataset,test)
        dataset,test=self.norm(dataset,test)
        
        return dataset,test,labels
        

In [3]:
dataset,test,labels=data_processing("C://Users/86155/Desktop/train.csv","C://Users/86155/Desktop/testA.csv").load()

In [4]:
dataset=torch.tensor(dataset,dtype=torch.float32)
labels=torch.tensor(labels)
datasets=dataset.split(100,dim=0)
labels=labels.split(100)

In [5]:
test=test[:,1:]
test=torch.tensor(test,dtype=torch.float32).to('cuda')

# 模型

In [6]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.linear1=nn.Linear(45,64)
        self.linear2=nn.Linear(64,128)
        self.linear3=nn.Linear(128,128)
        self.linear4=nn.Linear(128,64)
        self.linear5=nn.Linear(64,2)


    def forward(self,x):

        x=self.linear1(x)
        x=F.tanh(x)
        x=self.linear2(x)
        x=F.tanh(x)
        x=self.linear3(x)
        x=F.tanh(x)
        x=self.linear4(x)
        x=F.tanh(x)
        x=self.linear5(x)
        #x=F.softmax(x)
        
        return x
        

In [7]:
net=network().to('cuda')
optimizer = optim.Adam(net.parameters(), lr=0.00001) #设置优化器和学习率

In [8]:
weight=torch.tensor([0.1,0.9],dtype=torch.float32,device='cuda')


In [9]:
def train(epoches,datasets,lables,weight):
    for i in range(epoches):
        print(i,end='\r')
        
        
        for j in range(len(datasets)):
            optimizer.zero_grad()
            inputs=datasets[j].type(torch.float32).to('cuda')
            label=labels[j].to('cuda')
            output=net(inputs)
            loss=F.cross_entropy(output,label,weight=weight)
            loss.backward()
            optimizer.step()

In [10]:
train(100,datasets,labels,weight)

C:\Users\86155\Anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [11]:
output=net(test[:,:])

tensor([0, 1, 1,  ..., 1, 1, 0], device='cuda:0')

In [12]:
outputs=torch.argmax(output,axis=1)

In [ ]:
outputs=outputs.to('cpu').numpy()

In [19]:
outputs=outputs.reshape(-1,1)

In [20]:
final=np.concatenate((np.array(pd.read_csv('C://Users/86155/Desktop/testA.csv').loc[:,['id']]),outputs),axis=1)

In [21]:
result_ = pd.DataFrame(final, columns=['Id','isDefault'])


In [22]:
result_.to_csv("./out.csv", encoding="gbk", index=False)